## 1. Setup and Imports

In [ ]:
import sys
import os

# 1. 設置 Protobuf 使用 Python 實現
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# 2. 關鍵修復：屏蔽 TensorFlow
# 這會防止 transformers 導入 tensorflow，從而避免 protobuf 版本衝突
sys.modules["tensorflow"] = None 

import gc
from collections import Counter
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla P100-PCIE-16GB
Number of GPUs: 1


In [8]:
# 遍歷 input 目錄，查看檔案結構
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/deberta-v3-large-offline/spm.model
/kaggle/input/deberta-v3-large-offline/config.json
/kaggle/input/deberta-v3-large-offline/tokenizer.json
/kaggle/input/deberta-v3-large-offline/tokenizer_config.json
/kaggle/input/deberta-v3-large-offline/model.safetensors
/kaggle/input/deberta-v3-large-offline/special_tokens_map.json
/kaggle/input/deberta-v3-large-offline/added_tokens.json
/kaggle/input/deberta-v3-large-quest-qa-5fold/training_history_fold5.json
/kaggle/input/deberta-v3-large-quest-qa-5fold/training_history_fold1.json
/kaggle/input/deberta-v3-large-quest-qa-5fold/training_summary.json
/kaggle/input/deberta-v3-large-quest-qa-5fold/best_model_fold5.pth
/kaggle/input/deberta-v3-large-quest-qa-5fold/training_history_fold3.json
/kaggle/input/deberta-v3-large-quest-qa-5fold/best_model_fold2.pth
/kaggle/input/deberta-v3-large-q

## 2. Configuration

In [9]:
# ==========================================
# Configuration
# ==========================================
class Config:
    """Inference configuration"""
    # Model configuration
    model_name = "/kaggle/input/deberta-v3-large-offline"
    max_len = 512
    batch_size = 16  # Can increase for inference (16-32 depending on GPU memory)
    num_workers = 4
    seed = 42
    
    # Paths
    train_csv = "/kaggle/input/google-quest-challenge/train.csv"
    test_csv = "/kaggle/input/google-quest-challenge/test.csv"
    model_dir = "/kaggle/input/deberta-v3-large-quest-qa-5fold"
    output_dir = "/kaggle/working/"
    
    # Model files (5 folds)
    n_folds = 5
    model_files = [f"best_model_fold{i}.pth" for i in range(1, n_folds + 1)]
    
    # Target columns (30 labels)
    target_cols = [
        'question_asker_intent_understanding', 'question_body_critical', 'question_conversational',
        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent',
        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
        'question_type_compare', 'question_type_consequence', 'question_type_definition',
        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written',
        'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
        'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure',
        'answer_type_reason_explanation', 'answer_well_written'
    ]

def seed_everything(seed=42):
    """Set random seeds for reproducibility"""
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(Config.seed)

# Display configuration
print("✅ Configuration loaded successfully!\n")
print(f"🤖 Model: {Config.model_name}")
print(f"📏 Max Length: {Config.max_len}")
print(f"📦 Batch Size: {Config.batch_size}")
print(f"🎲 Random Seed: {Config.seed}")
print(f"🔢 Number of Folds: {Config.n_folds}")
print(f"\n📊 Number of target labels: {len(Config.target_cols)}")
print(f"\n📂 Model Directory: {Config.model_dir}")
print(f"📤 Output Directory: {Config.output_dir}")

✅ Configuration loaded successfully!

🤖 Model: /kaggle/input/deberta-v3-large-offline
📏 Max Length: 512
📦 Batch Size: 16
🎲 Random Seed: 42
🔢 Number of Folds: 5

📊 Number of target labels: 30

📂 Model Directory: /kaggle/input/deberta-v3-large-quest-qa-5fold
📤 Output Directory: /kaggle/working/


## 3. Dataset Class

In [10]:
# ==========================================
# Dataset Class
# ==========================================
class QuestTestDataset(Dataset):
    """Custom dataset for Q&A labeling inference"""
    
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        self.titles = df['question_title'].values
        self.bodies = df['question_body'].values
        self.answers = df['answer'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        title = str(self.titles[idx])
        body = str(self.bodies[idx])
        answer = str(self.answers[idx])
        
        # Combine question parts: [CLS] title [SEP] body [SEP] answer [SEP]
        q_text = title + " " + self.tokenizer.sep_token + " " + body
        a_text = answer
        
        # Tokenize
        q_tokens = self.tokenizer.tokenize(q_text)
        a_tokens = self.tokenizer.tokenize(a_text)
        
        # Dynamic truncation with budget awareness
        budget = self.max_len - 3  # Reserve for [CLS], [SEP], [SEP]
        if len(q_tokens) + len(a_tokens) > budget:
            half = budget // 2
            if len(a_tokens) > half and len(q_tokens) > half:
                # Both exceed half budget: truncate both
                a_tokens = a_tokens[:half]
                q_tokens = q_tokens[:budget - len(a_tokens)]
            elif len(a_tokens) <= half:
                # Answer fits in half: keep full answer, truncate question
                q_tokens = q_tokens[:budget - len(a_tokens)]
            else:
                # Question fits in half: keep full question, truncate answer
                a_tokens = a_tokens[:budget - len(q_tokens)]
                
        # Build input IDs: [CLS] + question_tokens + [SEP] + answer_tokens + [SEP]
        ids = [self.tokenizer.cls_token_id] + \
              self.tokenizer.convert_tokens_to_ids(q_tokens) + \
              [self.tokenizer.sep_token_id] + \
              self.tokenizer.convert_tokens_to_ids(a_tokens) + \
              [self.tokenizer.sep_token_id]
              
        mask = [1] * len(ids)
        padding_len = self.max_len - len(ids)
        ids = ids + [self.tokenizer.pad_token_id] * padding_len
        mask = mask + [0] * padding_len
        
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long)
        }

print("✅ Dataset class defined successfully!")

✅ Dataset class defined successfully!


## 4. Model Architecture

In [11]:
# ==========================================
# Model Class
# ==========================================
class QuestDebertaModel(nn.Module):
    """DeBERTa model with weighted layer pooling and multi-sample dropout"""
    
    def __init__(self, model_name=Config.model_name, num_labels=30):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.output_hidden_states = True
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        
        # Weighted layer pooling
        # Combines all hidden layers with learnable weights
        n_weights = self.config.num_hidden_layers + 1
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3  # Lower weight for earlier layers
        weights_init.data[-1] = 0    # Higher weight for last layer
        self.layer_weights = nn.Parameter(weights_init)
        
        # Multi-sample dropout (5 dropouts with 0.5 rate)
        # Averages predictions from 5 different dropout masks
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])
        self.fc = nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        # Get all hidden states
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states 
        
        # Extract [CLS] token from each layer: (batch, num_layers, hidden_size)
        cls_outputs = torch.stack([layer[:, 0, :] for layer in hidden_states], dim=1)
        
        # Apply weighted sum across layers
        weights = torch.softmax(self.layer_weights, dim=0).view(1, -1, 1)
        weighted_cls = (weights * cls_outputs).sum(dim=1)  # (batch, hidden_size)
        
        # Multi-sample dropout: average predictions from 5 different dropout masks
        logits_list = []
        for dropout in self.dropouts:
            logits_list.append(self.fc(dropout(weighted_cls)))
        avg_logits = torch.mean(torch.stack(logits_list, dim=0), dim=0)
        
        # Return logits (sigmoid will be applied during inference)
        return avg_logits

print("✅ Model class defined successfully!")

✅ Model class defined successfully!


## 5. Post-Processing Utilities

In [12]:
# ==========================================
# Post-processing Utilities (Winning Solution)
# ==========================================
def postprocess_single_column(target, ref):
    """
    Match the distribution of predicted column to training distribution.
    
    This technique from the winning solution adjusts predictions to follow
    the same distribution as the training data. Since Spearman correlation
    is rank-based, this can improve rankings.
    
    Args:
        target: Predicted values for a single column (numpy array)
        ref: Training values for the same column (numpy array)
        
    Returns:
        Postprocessed predictions scaled to [0, 1]
    """
    # Sort indices by predicted values
    ids = np.argsort(target)
    
    # Get value counts from training data, sorted by value
    counts = sorted(Counter(ref).items(), key=lambda s: s[0])
    scores = np.zeros_like(target)
    
    last_pos = 0
    v = 0
    
    # Assign rank values based on training distribution
    for value, count in counts:
        # Calculate position proportional to training distribution
        next_pos = last_pos + int(round(count / len(ref) * len(target)))
        if next_pos == last_pos:
            next_pos += 1
            
        # Assign same score to samples in this range
        cond = ids[last_pos:next_pos]
        scores[cond] = v
        last_pos = next_pos
        v += 1
    
    # Normalize to [0, 1]
    if scores.max() > 0:
        return scores / scores.max()
    return scores


def postprocess_predictions(predictions, train_df, target_cols):
    """
    Apply distribution matching and normalization to predictions.
    
    Args:
        predictions: numpy array of shape (n_samples, n_targets)
        train_df: Training dataframe with target columns
        target_cols: List of target column names
        
    Returns:
        Postprocessed predictions as numpy array
    """
    postprocessed = predictions.copy()
    
    # Columns where distribution matching showed improvement
    # (identified from winning solution's experiments)
    distribution_matching_cols = {
        'question_conversational',
        'question_type_compare',
        'question_type_definition',
        'question_type_entity',
        'question_has_commonly_accepted_answer',
        'question_type_consequence',
        'question_type_spelling',
        'question_type_choice',
        'question_not_really_a_question',
        'question_multi_intent',
        'question_type_procedure',
        'question_type_instructions',
        'answer_type_procedure',
        'answer_type_instructions',
        'question_expect_short_answer',
        'answer_type_reason_explanation',
    }
    
    for i, col in enumerate(target_cols):
        if col in distribution_matching_cols:
            # Apply distribution matching for specific columns
            postprocessed[:, i] = postprocess_single_column(
                postprocessed[:, i], 
                train_df[col].values
            )
        
        # Scale all columns to [0, 1] interval
        v = postprocessed[:, i]
        v_min, v_max = v.min(), v.max()
        if v_max > v_min:
            postprocessed[:, i] = (v - v_min) / (v_max - v_min)
        else:
            postprocessed[:, i] = 0.5  # If all values are the same
    
    return postprocessed

print("✅ Post-processing utilities defined successfully!")

✅ Post-processing utilities defined successfully!


## 6. Load Test Data

In [13]:
# ==========================================
# Load Test Data
# ==========================================
print("Loading test data...")
test_df = pd.read_csv(Config.test_csv)
print(f"✅ Test samples: {len(test_df):,}")
print(f"\nTest data columns: {list(test_df.columns)}")
print(f"\nFirst few rows:")
test_df.head()

Loading test data...
✅ Test samples: 476

Test data columns: ['qa_id', 'question_title', 'question_body', 'question_user_name', 'question_user_page', 'answer', 'answer_user_name', 'answer_user_page', 'url', 'category', 'host']

First few rows:


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


## 7. Prepare Test DataLoader

In [14]:
# ==========================================
# Prepare Test DataLoader
# ==========================================
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
print(f"✅ Tokenizer loaded: {tokenizer.__class__.__name__}")

print("\nCreating test dataset...")
test_dataset = QuestTestDataset(test_df, tokenizer, max_len=Config.max_len)
print(f"✅ Dataset size: {len(test_dataset)}")

print("\nCreating test dataloader...")
test_loader = DataLoader(
    test_dataset, 
    batch_size=Config.batch_size, 
    shuffle=False, 
    num_workers=Config.num_workers,
    pin_memory=True
)
print(f"✅ DataLoader created with batch_size={Config.batch_size}")
print(f"   Total batches: {len(test_loader)}")

Loading tokenizer...
✅ Tokenizer loaded: DebertaV2TokenizerFast

Creating test dataset...
✅ Dataset size: 476

Creating test dataloader...
✅ DataLoader created with batch_size=16
   Total batches: 30


## 8. Inference Function

In [15]:
# ==========================================
# Inference Function
# ==========================================
@torch.no_grad()
def generate_predictions(model, test_loader, device):
    """Generate predictions on test set"""
    model.eval()
    all_preds = []
    
    for batch in tqdm(test_loader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        
        # Get model outputs (logits)
        outputs = model(input_ids, mask)
        
        # Apply sigmoid to get probabilities
        outputs = torch.sigmoid(outputs)
        
        all_preds.append(outputs.cpu().numpy())
            
    return np.concatenate(all_preds)

print("✅ Inference function defined successfully!")

✅ Inference function defined successfully!


## 9. Run 5-Fold Ensemble Inference

In [ ]:
# ==========================================
# 5-Fold Ensemble Inference
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")

# Store predictions from each fold
fold_preds = []

print(f"{'='*60}")
print(f"Running 5-Fold Ensemble Inference")
print(f"{'='*60}\n")

for fold in range(1, Config.n_folds + 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{Config.n_folds}")
    print(f"{'='*60}")
    
    # Model path
    model_path = os.path.join(Config.model_dir, f"best_model_fold{fold}.pth")
    
    if not os.path.exists(model_path):
        print(f"⚠️  Model not found: {model_path}")
        continue
    
    print(f"Loading model from: {model_path}")
    
    # Initialize model
    model = QuestDebertaModel(
        model_name=Config.model_name,
        num_labels=len(Config.target_cols)
    )
    
    # Load weights
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully")
    
    # Generate predictions
    print(f"\nGenerating predictions for fold {fold}...")
    preds = generate_predictions(model, test_loader, device)
    fold_preds.append(preds)
    
    print(f"✅ Predictions shape: {preds.shape}")
    print(f"   Min: {preds.min():.4f}, Max: {preds.max():.4f}, Mean: {preds.mean():.4f}")
    
    # Clean up GPU memory
    del model, state_dict
    torch.cuda.empty_cache()
    gc.collect()
    
    if torch.cuda.is_available():
        print(f"   GPU Memory Used: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

print(f"\n{'='*60}")
print(f"All {len(fold_preds)} fold models processed successfully!")
print(f"{'='*60}")

Using device: cuda
GPU Memory: 17.06 GB

Running 5-Fold Ensemble Inference


Fold 1/5
Loading model from: /kaggle/input/deberta-v3-large-quest-qa-5fold/best_model_fold1.pth


2025-11-27 06:25:23.746439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764224723.948137      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764224724.005097      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## 10. Ensemble Predictions (Average)

In [ ]:
# ==========================================
# Ensemble Predictions
# ==========================================
print("\nAveraging predictions across folds...")
avg_preds = np.mean(fold_preds, axis=0)

print(f"✅ Ensemble predictions shape: {avg_preds.shape}")
print(f"   Min: {avg_preds.min():.4f}")
print(f"   Max: {avg_preds.max():.4f}")
print(f"   Mean: {avg_preds.mean():.4f}")
print(f"   Std: {avg_preds.std():.4f}")

# Display prediction statistics for each fold
print(f"\nPer-fold statistics:")
for i, preds in enumerate(fold_preds, 1):
    print(f"  Fold {i}: Mean={preds.mean():.4f}, Std={preds.std():.4f}")

## 11. Apply Post-Processing

In [ ]:
# ==========================================
# Apply Post-Processing
# ==========================================
print("\nApplying post-processing with distribution matching...")

# Load training data for distribution matching
if os.path.exists(Config.train_csv):
    train_df = pd.read_csv(Config.train_csv)
    print(f"✅ Training data loaded: {len(train_df)} samples")
    
    # Apply post-processing
    final_preds = postprocess_predictions(
        avg_preds, 
        train_df, 
        Config.target_cols
    )
    
    print(f"✅ Distribution matching applied to 16 selected columns")
    print(f"\nPost-processed predictions:")
    print(f"   Min: {final_preds.min():.4f}")
    print(f"   Max: {final_preds.max():.4f}")
    print(f"   Mean: {final_preds.mean():.4f}")
    print(f"   Std: {final_preds.std():.4f}")
else:
    print(f"⚠️  Training data not found at {Config.train_csv}")
    print(f"   Skipping distribution matching post-processing")
    final_preds = avg_preds

## 12. Create Submission File

In [ ]:
# ==========================================
# Create Submission File
# ==========================================
print("\nCreating submission file...")

# Create DataFrame with predictions
submission = pd.DataFrame(final_preds, columns=Config.target_cols)

# Add qa_id column
submission['qa_id'] = test_df['qa_id'].values

# Reorder columns: qa_id first, then all target columns
submission = submission[['qa_id'] + Config.target_cols]

print(f"✅ Submission DataFrame created")
print(f"   Shape: {submission.shape}")
print(f"   Columns: {len(submission.columns)}")
print(f"\nFirst few rows:")
submission.head()

In [ ]:
    # Create submission
    print("\nCreating submission file...")
    submission = pd.DataFrame(final_preds, columns=Config.target_cols)
    submission['qa_id'] = test_df['qa_id'].values
    submission = submission[['qa_id'] + Config.target_cols]
    
    # Save to CSV
    output_dir = "/kaggle/working"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "submission.csv")
    
    submission.to_csv(output_path, index=False)
    print(f"\n✅ Submission saved to: {output_path}")
    print(f"   File size: {os.path.getsize(output_path) / 1024:.2f} KB")
    
    # Verify submission format
    print(f"\n{'='*60}")
    print(f"Submission Verification")
    print(f"{'='*60}")
    print(f"✓ Number of rows: {len(submission)}")
    print(f"✓ Number of columns: {len(submission.columns)}")
    print(f"✓ qa_id column present: {'qa_id' in submission.columns}")
    print(f"✓ All target columns present: {all(col in submission.columns for col in Config.target_cols)}")
    print(f"✓ No missing values: {submission.isnull().sum().sum() == 0}")
    print(f"✓ All values in [0, 1]: {(submission[Config.target_cols].min().min() >= 0) and (submission[Config.target_cols].max().max() <= 1)}")
    print(f"\n✅ Submission file is ready for upload!")
    
    return submission

## 13. Summary Statistics

In [ ]:
# ==========================================
# Summary Statistics
# ==========================================
print(f"\n{'='*60}")
print(f"Inference Summary")
print(f"{'='*60}")
print(f"Model: microsoft/deberta-v3-large")
print(f"Number of folds: {len(fold_preds)}")
print(f"Test samples: {len(test_df)}")
print(f"Target labels: {len(Config.target_cols)}")
print(f"Post-processing: Distribution matching applied")
print(f"\nPrediction statistics per target:")

# Show statistics for each target
stats_df = pd.DataFrame({
    'Target': Config.target_cols,
    'Min': final_preds.min(axis=0),
    'Max': final_preds.max(axis=0),
    'Mean': final_preds.mean(axis=0),
    'Std': final_preds.std(axis=0)
})

stats_df

In [ ]:
# Display final statistics
print(f"\n{'='*60}")
print(f"✅ INFERENCE COMPLETED SUCCESSFULLY!")
print(f"{'='*60}")
print(f"📁 Output Directory: {Config.output_dir}")
print(f"📊 Ready for submission!")
print(f"{'='*60}")